In [1]:
import pandas as pd
import geopandas as gpd
import pathlib
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString, MultiPoint

import geojson
import h3

from geovoronoi import voronoi_regions_from_coords
crs4326 = {'init': 'epsg:4326'}

In [2]:
DIR_ROOT = os.path.join(pathlib.Path().absolute(), '../../..' )

influences_df = pd.read_csv(DIR_ROOT + '/data/04_itinerary/influences.csv')
influences_df = influences_df[influences_df['domestic'] == True]

localities = pd.read_csv(DIR_ROOT + '/data/02_processed/localities.csv')
localities['geometry'] = localities['geometry'].apply(wkt.loads)
localities = gpd.GeoDataFrame(localities, crs=crs4326).set_geometry('geometry')

/home/casus-fwu036/anaconda3/envs/po2/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [3]:
bbox = localities.buffer(1).total_bounds
bounds = dict(geojson.Polygon([[[bbox[0], bbox[1]], [bbox[0], bbox[3]], [bbox[2], bbox[3]], [bbox[2], bbox[1]]]]))
hex_ids = list(h3.polyfill(bounds, 4))

hex_df = gpd.GeoDataFrame(
    [{"geometry": Polygon(h3.h3_to_geo_boundary(x)), "id": x} for x in hex_ids], crs=crs4326
)
hex_df.set_index('id', inplace=True)

hex_df['id'] = hex_df.index
hex_df['geometry'] = hex_df.apply(lambda x: Polygon(h3.h3_to_geo_boundary(x['id'])), axis=1)
hex_df

<ipython-input-3-7dfe25777e9d>:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bbox = localities.buffer(1).total_bounds


,geometry,id
id,,
8463a4bffffffff,"POLYGON ((15.20844 52.68263, 15.02006 52.55990...",8463a4bffffffff
8452c63ffffffff,"POLYGON ((12.41131 45.06760, 12.21167 44.94877...",8452c63ffffffff
844394dffffffff,"POLYGON ((20.50239 54.11574, 20.30748 54.19866...",844394dffffffff
847a817ffffffff,"POLYGON ((6.27104 47.88829, 6.46855 47.95450, ...",847a817ffffffff
8452ec9ffffffff,"POLYGON ((15.25890 47.87746, 15.06071 47.75458...",8452ec9ffffffff
...,...,...
845258dffffffff,"POLYGON ((15.52733 50.18800, 15.33344 50.06460...",845258dffffffff
847a803ffffffff,"POLYGON ((5.96309 48.61016, 6.16053 48.67522, ...",847a803ffffffff
8452e13ffffffff,"POLYGON ((14.84306 47.18684, 14.64402 47.06459...",8452e13ffffffff


In [10]:
def get_influence_origins(hex_geo):
    localities_in_hex = localities[localities.intersects(hex_geo)]

    infl_list = []
    for lihi, locality_in_hex in localities_in_hex.iterrows():
        inf_object_rows = influences_df[influences_df['name'] == locality_in_hex['name']]
        infl_list.extend(inf_object_rows['locality'].tolist())

    return { x: infl_list.count(x) for x in infl_list}

hex_df['infl'] = hex_df.apply(
    lambda x: get_influence_origins(x['geometry']), 
    axis=1
)
print(hex_df['infl'].tolist())

hex_df['all'] = hex_df.apply(
    lambda x: sum(x['infl'].values()), 
    axis=1
)

centers_to_check = ['Praha', 'Brno', 'Wien', 'Graz', 'Opava', 'Linz', 'Kostanjevica na Krki', 'Friesach']


for center in centers_to_check:
    hex_df[center] = hex_df.apply(
        lambda x: x['infl'][center] if center in x['infl'] else 0, 
        axis=1
    )

hex_df['others'] = hex_df.apply(
    lambda x: x['all'] - sum(x[center] for center in centers_to_check) , 
    axis=1
)



[{}, {}, {}, {}, {'Znojmo': 2, 'Brno': 1, 'Korneuburg': 1, 'Wien': 2, 'Písek': 1, 'Praha': 1}, {'Graz': 4, 'Velehrad': 1, 'Wien': 5, 'Brno': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Wien': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Wien': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Slovenj Gradec': 3, 'Wien': 1, 'Graz': 1, 'Praha': 1}, {}, {}, {}, {'Korneuburg': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Brno': 3, 'Zbraslav': 1, 'Wien': 1, 'Krems an der Donau': 1, 'Znojmo': 1, 'Wiener Neustadt': 1}, {}, {}, {'Praha': 7}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Poděbrady': 1, 'Praha': 1}, {}, {}, {}, {}, {}, {}, {}, {'Wien': 9, 'Graz': 4, 'Jihlava': 1, 'Głubczyce': 1, 'Freistadt': 3, 'Linz': 2, 'Znojmo': 2, 'Brno': 5, 'Písek': 1}, {}, {}, {}, {}, {}, {}, {'Wiener Neustadt': 1, 'Wien': 1, 'Graz': 1, 'Velehrad': 2, 'Praha': 1}, {}, 

In [11]:

hex_df.reset_index(drop=True).to_file(DIR_ROOT + '/data/06_outputs/hex_influence.geojson', driver="GeoJSON")

hex_df_geo = hex_df.copy()
hex_df_geo['centroid'] = hex_df_geo.apply(
    lambda x: x['geometry'].centroid, 
    axis=1
)
hex_df_geo = hex_df_geo.drop(columns=["geometry"])
hex_df_geo.set_geometry('centroid', inplace=True)
hex_df_geo = hex_df_geo.reset_index(drop=True)
print(hex_df_geo)
hex_df_geo.to_file(DIR_ROOT + '/data/06_outputs/hex-centroids_influence.geojson', driver="GeoJSON")


                   id                                               infl  all  \
0     8463a4bffffffff                                                 {}    0   
1     8452c63ffffffff                                                 {}    0   
2     844394dffffffff                                                 {}    0   
3     847a817ffffffff                                                 {}    0   
4     8452ec9ffffffff  {'Znojmo': 2, 'Brno': 1, 'Korneuburg': 1, 'Wie...    8   
...               ...                                                ...  ...   
1973  845258dffffffff                                                 {}    0   
1974  847a803ffffffff                                                 {}    0   
1975  8452e13ffffffff  {'Graz': 3, 'Wien': 9, 'Praha': 3, 'Velehrad':...   27   
1976  84520e3ffffffff                                                 {}    0   
1977  8452235ffffffff                                                 {}    0   

      Praha  Brno  Wien  Zn